## Instagram Comment Getter - *docker deployment*


- Selenium Web Scraping for IG Comments

In [1]:
import pandas as pd
import os

### Configurations for Docker container deployment

In [2]:
# path to repo decker deploy script
deploy_script_path = './deploy.sh' # <-- Make sure to edit for your docker mount -v

In [3]:
# IG Username (`username` in ig_df)
input_wwe_username = 'blackivystories'

In [4]:
# Our instagram df from PhantomBuster
ig_df = pd.read_csv('./data/blackivystories_PostsExtractor_03012021.csv')

### Loop through urls for BlackIvyStories IG username ...


In [5]:
# Our exported Azure OCR df with text
ocr_df = pd.read_json('./data/blackivystories_ig_ocr_expanded.json')

In [6]:
ocr_df.columns

Index(['index', 'language', 'text_angle', 'orientation', 'regions', 'filename',
       'txt', 'ivy'],
      dtype='object')

## Running for all URLs now

- though it might make sense to go one college at a time in case things fail...

### We want to get our Penn urls... this worked in the jhub deployment. What about dartmouth for docker deployment?

I am reverse extracting this from the filename... probably wasn't best strategy to start with

In [7]:
#ivy_input_name = 'penn'
#ivy_input_name = 'dartmouth'

In [8]:
postid_filename = []
ivy_ocr_pid = []

for pid in ocr_df[(ocr_df.ivy == 'brown') |
                  (ocr_df.ivy == 'princeton') |
                  (ocr_df.ivy == 'harvard') |
                  (ocr_df.ivy == 'columbia') |
                  (ocr_df.ivy == 'yale') |
                  (ocr_df.ivy == 'cornell')|
                  (ocr_df.ivy == 'penn') |
                  (ocr_df.ivy == 'dartmouth')].filename.str[:-4]:
    ivy_ocr_pid.append(int(pid))
    
ivy_ig_post_urls = ig_df[ig_df.postId.isin(ivy_ocr_pid)].postUrl.unique()


#### *TOTAL NUMBER OF URLS FOR IVY IG STORIES*

In [9]:
len(ivy_ig_post_urls)

318

In [10]:
# get our ids
url_ids = []

for url in ivy_ig_post_urls:
    url_ids.append(url.split('/')[-2])
    
# check if they already ran (in case the cell fails)
url_for_commentgetter = []

for ig_url in url_ids:
    json_outfile = ig_url + '.json'
    if json_outfile not in os.listdir('./data/etl/'): # <-- the url doesn't already have a json etl outfile
        url_for_commentgetter.append('https://www.instagram.com/p/{}/'.format(ig_url))

#### *HOW MANY URLS ARE YOU DEPLOYING DOCKER COMMENT GETTER FOR:*

*--> If its zero then you are done for that respective ivy filter!*

In [11]:
len(url_for_commentgetter)

180

__________

## DOCKER DEPLOYMENT FOR INSTAGRAM COMMENT GETTER


- *You need Docker installed on your host... more info here: https://docs.docker.com/get-docker/*

- I think Firefox is best for testing but in the past I remember using Chrome as it was more friendly in docker with selenium... we will see:

- I keep getting connection dropped for firefox so maybe chrome is best?

_________

```bash
docker run -d -p 4444:4444 -v --shm-mem=4G selenium/standalone-firefox
# you can rename this container

# check for running container name
docker ps
docker rename _whatever_random_docker_name_is_ selenium-firefox

# nice to have a terminal windows visible with docker stats for monitoring
docker stats
```

__________
#### *RUN FOR FIRST BUILD, OR JUST RESTART*

In [12]:
#!docker run -d -p 4444:4444 -v --shm-mem=4G selenium/standalone-chrome

In [13]:
!docker stop selenium-firefox

selenium-firefox


#### *--> Give your Docker container a moment to startup...*

In [14]:
!docker restart selenium-chrome

selenium-chrome


In [15]:
!docker ps

CONTAINER ID   IMAGE                        COMMAND                  CREATED       STATUS                  PORTS                    NAMES
cddff367b0f5   selenium/standalone-chrome   "/opt/bin/entry_poin…"   4 hours ago   Up Less than a second   0.0.0.0:4444->4444/tcp   selenium-chrome


## *Now running for the remaining ivys...*

- pushing to github first as it'll run overnight ...

- some of the containers fail which is alright, we can rerun after this batch completes. 
__________


### Looping through the 55 penn story instagram URLs for comment web scraping ... 

- tested on loop & deploy for the 16 dartmouth URLs before running for all


Network errors seem to mess this up... maybe the restart on failure flag isn't helpful? The pipeline failed to stop a restarted container...

- I removed `--restart=on-failure` from [deploy.sh](./deploy.sh)... I think if a container fails we can just rerun by refreshing urls for comment getter in cells above:

- there was a runaway container which hit 150+ clicks very quickly. I think I set a threshold of 500 clicks and it stops, I just hit the stop button to stop that container though. I probably need to rerun for just that one URL? I bumped this down to 100

### The `selenium-chrome` seems to work better than `selenium-firefox`

- the latter occassionally refuses the connection...

- not sure why but the following URL keeps running off & clicking for the following https://www.instagram.com/p/CBn-zFWjDIf/... it did get comments and export to json though

- the first error that chrome threw was the following (the firefox kept again sending connection refused):

```
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: headless chrome=85.0.4183.83)
```

which then becomes

```
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.WebDriverException: Message: chrome not reachable
```

in my terminal I tried to manually restart the selenium-chrome
- docker-compose for a one url to one browser would be convenient ...
this produced error:
```
rllib3.exceptions.ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
```

I thought to add `docker restart selenium-chrome` on deploy.sh but that started to yield straight 3 comment yields... I think the posts did not all have same number of comments. I deleted those files to run again...

_______

So why does this not get more than 3 comments? https://www.instagram.com/p/CDzKZlHhu0Z

```
Getting comments for IG post --> https://www.instagram.com/p/CDzKZlHhu0Z
commentGetter
commentGetter
Sending build context to Docker daemon  93.61MB
Step 1/6 : FROM python:3.8-slim
 ---> 62297c9f4e5c
Step 2/6 : COPY GetComments.py /app/GetComments.py
 ---> Using cache
 ---> 41fdc36d0ee7
Step 3/6 : COPY configs /app/configs
 ---> Using cache
 ---> 463216f43026
Step 4/6 : WORKDIR /app
 ---> Using cache
 ---> 152cf4ba995d
Step 5/6 : RUN pip install -r /app/configs/requirements.txt
 ---> Using cache
 ---> 6989b6deb091
Step 6/6 : CMD ["python3", "GetComments.py", "https://www.instagram.com/p/CDzKZlHhu0Z"]
 ---> Using cache
 ---> 53c39f6ab927
Successfully built 53c39f6ab927
Successfully tagged instagram_commentgetter:latest
------------------------------------------------------------
Raw html outfile prepared as: /data/raw/CDzKZlHhu0Z.html
ETL outfile prepared as: /data/etl/CDzKZlHhu0Z.json
Exporting to .html file --> /data/raw/CDzKZlHhu0Z.html
Done! Number of comments extracted --> 3
Exporting ETL json file --> /data/etl/CDzKZlHhu0Z.json
Complete!
```

In [ ]:
for url in url_for_commentgetter:
  
    ##################################################################
    # Clean up URL (in case lacking https or has trailing slash)...
    if not url.startswith('https://'):
        url = 'https://' + url
    if url.endswith('/'):
        url=url[:-1]
    
    ##################################################################
    # Prepare Dockerfile (using template to overwrite for next URL)
    dockerfile_in = open("./configs/dockerfile_template", "rt")
    dockerfile_out = open("./Dockerfile", "wt")
    
    # Replace the value w/ URL -- this loops through lines, would be better using regex...
    for line in dockerfile_in:
        # This is how url gets passed to container...
        dockerfile_out.write(line.replace('replace_with_url', '"{}"'.format(url)))
    
    #close input and output dockerfile & template
    dockerfile_in.close()
    dockerfile_out.close()
    
    ##################################################################
    # BEGIN LOOPING THROUGH URLS BY LAUNCHING DOCKER CommentGetters
    print('Getting comments for IG post --> {}'.format(url))
    try:
        !bash $deploy_script_path
    except:
        print('oops! failed for {}'.format(url))
        break

Getting comments for IG post --> https://www.instagram.com/p/CDzKZlHhu0Z
commentGetter
commentGetter
Sending build context to Docker daemon  93.61MB
Step 1/6 : FROM python:3.8-slim
 ---> 62297c9f4e5c
Step 2/6 : COPY GetComments.py /app/GetComments.py
 ---> Using cache
 ---> 41fdc36d0ee7
Step 3/6 : COPY configs /app/configs
 ---> Using cache
 ---> 463216f43026
Step 4/6 : WORKDIR /app
 ---> Using cache
 ---> 152cf4ba995d
Step 5/6 : RUN pip install -r /app/configs/requirements.txt
 ---> Using cache
 ---> 6989b6deb091
Step 6/6 : CMD ["python3", "GetComments.py", "https://www.instagram.com/p/CDzKZlHhu0Z"]
 ---> Using cache
 ---> 53c39f6ab927
Successfully built 53c39f6ab927
Successfully tagged instagram_commentgetter:latest
------------------------------------------------------------
Raw html outfile prepared as: /data/raw/CDzKZlHhu0Z.html
ETL outfile prepared as: /data/etl/CDzKZlHhu0Z.json
Exporting to .html file --> /data/raw/CDzKZlHhu0Z.html
Done! Number of comments extracted --> 3
Export

Exporting to .html file --> /data/raw/CDerlBZBn5S.html
Done! Number of comments extracted --> 3
Exporting ETL json file --> /data/etl/CDerlBZBn5S.json
Complete!
---------------------------------------------------------------------------
commentGetter
Getting comments for IG post --> https://www.instagram.com/p/CDee-KMBUJe
Error response from daemon: No such container: commentGetter
Error: No such container: commentGetter
Sending build context to Docker daemon  93.62MB
Step 1/6 : FROM python:3.8-slim
 ---> 62297c9f4e5c
Step 2/6 : COPY GetComments.py /app/GetComments.py
 ---> Using cache
 ---> 41fdc36d0ee7
Step 3/6 : COPY configs /app/configs
 ---> Using cache
 ---> 463216f43026
Step 4/6 : WORKDIR /app
 ---> Using cache
 ---> 152cf4ba995d
Step 5/6 : RUN pip install -r /app/configs/requirements.txt
 ---> Using cache
 ---> 6989b6deb091
Step 6/6 : CMD ["python3", "GetComments.py", "https://www.instagram.com/p/CDee-KMBUJe"]
 ---> Running in 9011624989ea
Removing intermediate container 901162

__________

In [17]:
!docker ps

CONTAINER ID   IMAGE                            COMMAND                  CREATED              STATUS              PORTS                    NAMES
cf28a2d6f945   instagram_commentgetter:latest   "python3 GetComments…"   About a minute ago   Up About a minute                            commentGetter
35faafb3499e   selenium/standalone-firefox      "/opt/bin/entry_poin…"   About an hour ago    Up 9 minutes        0.0.0.0:4444->4444/tcp   selenium-firefox


In [18]:
etl_files = [item for item in os.listdir('./data/etl/') if item.endswith('.json')]

### Post deployment -- Combine all etl JSON comments into df

In [19]:
etl_files = [item for item in os.listdir('./data/etl/') if item.endswith('.json')]

dfs = []

for etl in etl_files:
    dfs.append(pd.read_json('./data/etl/{}'.format(etl)))
    
etl_df = pd.concat(dfs)

In [20]:
etl_df.shape

(1109, 3)

In [21]:
etl_df

,url,author,comment
0,https://www.instagram.com/p/CB1vWGmJJWc,blackivystories,@uofpenn . . . . . . #blackstudentsmatter #b...
1,https://www.instagram.com/p/CB1vWGmJJWc,doreenm1,😡 @uofpenn @columbia enough of this racist...
2,https://www.instagram.com/p/CB1vWGmJJWc,jay_theorist,@skaijackson 36w Reply
3,https://www.instagram.com/p/CB1vWGmJJWc,View,replies (1)
4,https://www.instagram.com/p/CB1vWGmJJWc,janzibrown,Wow 36w 1 like Reply
...,...,...,...
24,https://www.instagram.com/p/CCl0jqGh0S1,View,replies (1)
25,https://www.instagram.com/p/CCl0jqGh0S1,victoriatellez,What the fuck 33w 1 like Reply
26,https://www.instagram.com/p/CCl0jqGh0S1,kiwikiwikiwi25,😞 What an ignorant and horrible thing to say. ...
27,https://www.instagram.com/p/CCl0jqGh0S1,catie.mc23,@dartmouthcollege 33w Reply
